# Chatbot + Recomanació

L'objectiu d'aquesta pràctica és força directe, volem recomanar a l'usuari noves `aisle_id` on anar a comprar quan ens ho demani.

Es dóna tant un chatbot funcional com un recomanador basat en factorització de matrius, però si voleu podeu fer servir les vostres implementacions pròpies del chatbot i del recomanador basat en pagerank.

Si feu servir el codi donat, llegiu les consideracions que trobareu més abaix

In [1]:
import zipfile
from os.path import join, dirname
import pandas as pd

def locate(*path):
    base = globals().get('__file__', '.')
    return join(dirname(base), *path)

def unzip(file):
    zip_ref = zipfile.ZipFile(locate(file), 'r')
    zip_ref.extractall(locate('data'))
    zip_ref.close()

unzip('order_products__train.csv.zip')
unzip('orders.csv.zip')
unzip('products.csv.zip')
unzip('aisles.csv.zip')

In [2]:
df_order_prods = pd.read_csv(locate('data', 'order_products__train.csv'))
df_orders = pd.read_csv(locate('data', 'orders.csv'))[['order_id', 'user_id']]
df_prods = pd.read_csv(locate('data', 'products.csv'))[['product_id', 'aisle_id']]

df_merged = pd.merge(pd.merge(df_order_prods, df_orders, on='order_id'), df_prods, on='product_id')
counts = df_merged.groupby(['user_id', 'aisle_id']).size()
df_counts = counts.unstack()

In [3]:
df_aisles = pd.read_csv(locate('data', 'aisles.csv'), index_col='aisle_id')

## Pràctica

**Punt 1.** L'usuari solament ha de poder afegir productes que es trobin en el dataframe `df_aisles`, és a dir, el nom del producte s'ha de trobar en la columna `aisles` d'aquest dataframe

**Punt 2.** Quan l'usuari enviï la comanda `/reco` i solament si es troba afegint productes (podeu cridar `user.is_adding()` per comprovar-ho), li heu de recomanar un nou producte que no tingui ja a la llista. Per fer-ho, els passos que seguirem seran els següents:

1. Buscar a la base de dades (`df_counts`) la persona més semblant a l'usuari del bot. Això es pot fer de diverses maneres, per exemple, pots mirar quina persona té una distància més petita respecte l'usuari tenint en compte les compres, amb `np.linalg.norm(compres_persona_db - llista_usuari)`, o agant la que té la correlació de pearson més gran entre les seves compres i la de l'usuari (`scipy.stats.stats.pearsonr` o el mètode `corr` dels dataframes).

Està clar, per fer això necessites la llista de productes afegits de l'usuari en funció de `aisle_id` (no el nostre `product_id`) i de la quantitat `qty`, pots obtenir-ho a partir de `user_info(user.id)['products']`.

2. Un cop tens aquesta persona, calcula el seu `score` (ie. l'estimació de compra) per totes les `aisle_id` que l'usuari no hagi comprat encara.

3. Envia un missatge a l'usuari amb el nom de la `aisle_id` que ha tret millor puntuació en el punt anterior i la puntuació arrodonida a l'enter més proper

## Consideracions del recomanador

El recomanador es dóna ja entrenat (arxius P.pkl i Q.pkl), però de forma ràpida i poc fiable. Podeu obtenir la recomanació d'un usuari de la base de dades (df_counts) per a un item (aisle_id) donat amb el mètode `estimate(usuari, item)`.

In [6]:
from recommender import NMFRecommender

reco = NMFRecommender(df_counts, 3, 10)
reco.factorize()
reco.estimate(1, 2)

4.9759486763891934

## Consideracions del chatbot

* No teniu accés directe al codi del bot, treballareu a partir de funcions "callback", és a dir, quan el bot detecta un event cridarà les vostres funcions. Les funcions, totes elles, tenen un dos paràmetres en comú:
    * `bot`: Objecte DelegatorBot de Telepot
    * `user`: Objecte ShoppingUser
    
Les funcions que es cridaran a mode de callback són:

* Quan es rep una comanda, és a dir un missatge que comença per /, es cridarà `on_cmd(bot, user, cmd)`. El paràmetre `cmd` conté la comanda enviada per l'usuari
* Quan s'afegeix un producte, es crida `on_add(bot, user, product_id, qty)`, on `product_id` indica el nom del producte i `qty` la quantitat comprada. Si aquesta funció retorna True o None, l'item s'afegirà a l'usuari, però si retorna False **no** s'afegirà. Sii esta a aisle_id
* Quan es marca un producte com a comprat (si encara no estava comprat), es crida `on_flag(bot, user, product_id)`
* Quan s'acaben de comprar tots els productes, i solament 1 cop per interacció, es crida `on_end(bot, user)`

**Els productes de l'usuari ja no són una llista de productes, sinó diccionari de productes**:

```python
{
    ...
    'products': {
        'product_id_1': {
            'status': 0/1,
            'qty': <int>
        },
        ...
        'product_id_n': {
            'status': 0/1,
            'qty': <int>
        },
    }
    ...
}
```

In [11]:
import numpy as np
from operator import itemgetter
from sklearn.preprocessing import Normalizer
from emoji import emojize

  
async def on_cmd(bot, user, cmd):
    if cmd == '/start':
        await bot.sendPhoto(user.id, open('img/hello.jpg', 'rb'))
    elif cmd == '/done':
        await user.sender.sendMessage('Let\'s go')
    elif cmd == '/reco':
        if user.is_adding():
            #Com que triga un pel a recomanar el producte, enviem aquest missatge al usuari
            await user.sender.sendMessage("Espera mentre miro quin producte et resultaria ideal! 🤔")
            
            #Vector on afegirem la quantitat dels productes que ha comprat l'usuari en la posicio adequada
            vector = np.zeros(df_aisles.size)
            
            #llista per guardar els productes comprats
            productes = []
            
            for i in user_info(user.id)['products']: #Per tots els productes comprats
                productes.append(i) #Els afegim a la llista 
                #Afegim la quantitat comprada d'aquest producte en la correcte posicio del vector
                vector[df_aisles[df_aisles['aisle']==i].index[0]] = user_info(user.id)['products'][i]['qty']
            
            #Creo un dataframe amb aquest vector. El vull en columnes aixi que el transposo
            dfVector = pd.DataFrame({'Vect':vector}).transpose() 
            
            #Ajuntu df_counts amb aquest dataframe. Trec els NaN per la correlacio
            df_Junt = pd.concat([df_counts,dfVector]).fillna(0)
            
            #Normalitzo tot el dataframe. Fit_transform perque no em retorni un array
            df_Normalitzat = Normalizer().fit_transform(df_Junt)
            
            #Calculo la correlacio entre tots els valors del Df
            correlacions = pd.DataFrame(df_Normalitzat).corr(method='pearson')
        
            #Pero a mi nomes m'interessa l'ultima fila
            correlacions_temp = correlacions.iloc[-1].fillna(0)
            
            #Aquesta fila la paso a vector
            correlacions_temp = correlacions_temp.values.tolist()
            
            #Per calcular el maxim, l'hi trec l'1 de la correlacio amb ell mateix. Hi poso un zero
            correlacions_temp[len(correlacions_temp)-1] = 0
            
            #Agafo la posicio del maxim
            persona = np.argmax(correlacions_temp)
            
            #Trobo els index dels items que ha comprat aquesta persona
            items_persona = df_counts.loc[persona].loc[df_counts.loc[persona] > 0 ].index
        
            puntuacions = [] #Llista per guardar les puntuacions
            for i in items_persona: #Per cada item de la persona semblant
                if df_aisles.loc[i].item() not in productes: #Sino ha estat comprat per el nostre usuari
                    score = reco.estimate(persona, i) #Fem la estimació i l'afegim a la llista
                    puntuacions.append((i,score))
            index_item = max(puntuacions,key=itemgetter(1))[0] #Agafem el maxim d'aquesta llista
            
            #L'index de l'item amb puntuacio maxima. Hem quedo amb el seu nom amb .item()
            item = df_aisles.loc[index_item].item()
            await user.sender.sendMessage("Et recomano que compris: "+item+" 😋") #L'enviem per missatge
            
            # Triga uns 5 segons aproximadament en fer la recomanació, creiem que és per el mètode que fem servir per 
            # calcular les correlacions.
        else:
            await user.sender.sendMessage("Si no estàs afegint no et puc recomanar res!😪")
            
async def on_add(bot, user, product_id, qty):
    #Mirem si a df_aisles hi ha el seu producte. .Any() retorna True o False
    return ((df_aisles['aisle'] == product_id).any())   
           
async def on_flag(bot, user, product_id):
    pass
    
async def on_end(bot, user):
    await bot.sendPhoto(user.id, open('img/done.png', 'rb'))
    pass

In [ ]:
if __name__ == '__main__':
    from botBlaiSergi import ShoppingBot, user_info
    
    with ShoppingBot() as bot:
        # Setup callbacks
        bot.add_callback('cmd', on_cmd)
        bot.add_callback('add-product', on_add)
        bot.add_callback('flag-product', on_flag)
        bot.add_callback('end', on_end)
        
        # Start bot
        bot.start(open('TOKEN').read().strip())